In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define your data directory
data_dir = '/kaggle/input/swimcat/Swimcat'

# Create an ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255
)

# Load the training dataset
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Adjust the target size according to your model requirements
    batch_size=32,
    class_mode='categorical',  # 'binary' for binary classification
)


datagen_1 = ImageDataGenerator(rescale=1./255)
test_generator = datagen_1.flow_from_directory(
    '/kaggle/input/swimcat/Swimcat_test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/swimcat/Swimcat'

In [ ]:

import numpy as np
def load(data):
    train_x=[]
    train_y=[]
    for i in range(len(data)):
        image,label=data[i]
        train_x.extend(image)
        train_y.extend(label)
    return np.array(train_x),np.array(train_y)


In [ ]:
train_x,train_y=load(train_generator)
test_x,test_y=load(test_generator)

In [ ]:
def make_it_binary(prediction_arr):
    for arr in prediction_arr:
        max_index = np.argmax(arr)
        arr[:] = 0
        arr[max_index] = 1
    return prediction_arr

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
def evaluate(test_p,actual,name='model',all=True):
    test=make_it_binary(test_p)
    print(f'{name} accuracy: {accuracy_score(actual,test)}\n')
    if all==True:
        for i in range(6):
            print(f'{name} accuracy for class {i+1}: {accuracy_score(actual[:,i],test[:,i])}')
            print(f'{name} precision for class {i+1}: {precision_score(actual[:,i],test[:,i])}')
            print(f'{name} recall for class {i+1}: {recall_score(actual[:,i],test[:,i])}')
            print(f'{name} f1 score for class {i+1}: {f1_score(actual[:,i],test[:,i])}\n')
    

In [ ]:
def class_graph(test_p,actual):
    import matplotlib.pyplot as plt
    test=make_it_binary(test_p)
    class_accuracy=[]
    class_f1=[]
    class_name=[]
    for i in range(6):
        class_accuracy.append(accuracy_score(actual[:,i],test[:,i]))
        class_f1.append(f1_score(actual[:,i],test[:,i]))
        class_name.append(f'class {i+1}')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Subplot 1: Accuracy
    ax1.plot(class_name, class_accuracy, marker='o', linestyle='-', color='b', label='Accuracy')
    ax1.set_title('Accuracy for Each Class')
    ax1.set_xlabel('Class Names')
    ax1.set_ylabel('Accuracy')
    ax1.set_xticklabels(class_name, rotation=45, ha='right')  # Rotate x-axis labels for better readability
    ax1.grid(True)
    ax1.legend()

    # Annotate each point with its accuracy value
    for i, txt in enumerate(class_accuracy):
        ax1.annotate(f'{txt:.2f}', (class_name[i], class_accuracy[i]), textcoords="offset points", xytext=(0,5), ha='center')

    # Subplot 2: F1 Score
    ax2.plot(class_name, class_f1, marker='o', linestyle='-', color='r', label='F1 Score')
    ax2.set_title('F1 Score for Each Class')
    ax2.set_xlabel('Class Names')
    ax2.set_ylabel('F1 Score')
    ax2.set_xticklabels(class_name, rotation=45, ha='right')  # Rotate x-axis labels for better readability
    ax2.grid(True)
    ax2.legend()

    # Annotate each point with its F1 score value
    for i, txt in enumerate(class_f1):
        ax2.annotate(f'{txt:.2f}', (class_name[i], class_f1[i]), textcoords="offset points", xytext=(0,5), ha='center')

    # Adjust layout to prevent clipping of x-axis labels
    plt.tight_layout()

    # Show the plot
    plt.show()


## vgg16 model

In [ ]:

from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load pre-trained AlexNet model without top (final fully connected layers)
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for your specific task
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(6, activation='softmax')  # num_classes is the number of classes in your task
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

model.fit(train_x,train_y,epochs=30)


In [ ]:
prediction=model.predict(train_x)
test_prediction=model.predict(test_x)


evaluate(prediction,train_y,'train')
evaluate(test_prediction,test_y,'test')



# non-aug accuracy
# train accuracy: 0.996268656716418
# test accuracy: 0.9654471544715447



In [ ]:
class_graph(test_prediction,test_y)

## resnet 50 model

In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50V2 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout, MaxPooling2D,Conv2D, Activation, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam

# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = ResNet50V2(weights='imagenet', include_top=False,input_shape=(224,224,3))

# Add a custom top (fully connected) layer for your specific task
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x=  Dropout(0.4)(x)
predictions = Dense(6, activation='softmax')(x)  # Adjust 'num_classes' based on your task

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# Freeze the weights of the pre-trained layers

# Compile the model with an appropriate optimizer, loss, and metrics
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_x, train_y, epochs=30)


In [ ]:

test_prediction=model.predict(test_x)
prediction=model.predict(train_x)

evaluate(prediction,train_y,'train',False)
evaluate(test_prediction,test_y,'test',False)

# non-aug:
# train accuracy: 1.0
# test accuracy: 0.9227642276422764

In [ ]:
# model.save('/kaggle/working/res_net_50.h5')
# test_prediction=model.predict(test_x)
# class_graph(test_prediction,test_y)

## inception model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
# from tensorflow.keras.applications import AlexNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pre-trained AlexNet model without top (final fully connected layers)
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for your specific task
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(6, activation='softmax')  # num_classes is the number of classes in your task
])

# Compile the model
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

model.fit(train_x,train_y,epochs=35)

In [ ]:
prediction=model.predict(train_x)
test_prediction=model.predict(test_x)


evaluate(prediction,train_y,'train',False)
evaluate(test_prediction,test_y,'test',False)

# for non-aug
# train accuracy: 0.9981343283582089
# test accuracy: 0.9126016260162602

In [ ]:
# model.save('/kaggle/working/inception.h5')
test_prediction=model.predict(test_x)
class_graph(test_prediction,test_y)

## mobilenet

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 with pre-trained weights (excluding the top classification layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model for fine-tuning
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.4)(x)
predictions = layers.Dense(6, activation='softmax')(x)

# Combine the base model with the custom top model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

In [ ]:
model.fit(train_x,train_y,epochs=35)

In [ ]:
prediction=model.predict(train_x)
test_prediction=model.predict(test_x)


evaluate(prediction,train_y,'train',False)
evaluate(test_prediction,test_y,'test',False)

# for non-aug:
# train accuracy: 0.9819651741293532
# test accuracy: 0.9288617886178862


In [ ]:
# model.save('/kaggle/working/mobilenet.h5')
test_prediction=model.predict(test_x)
class_graph(test_prediction,test_y)

## densenet

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load DenseNet121 with pre-trained weights (excluding the top classification layer)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model for fine-tuning
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.4)(x)
predictions = layers.Dense(6, activation='softmax')(x)

# Combine the base model with the custom top model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
# Display the model summary
model.summary()

In [ ]:
model.fit(train_x,train_y,epochs=30)

In [ ]:
prediction=model.predict(train_x)
test_prediction=model.predict(test_x)


evaluate(prediction,train_y,'train',False)
evaluate(test_prediction,test_y,'test',False)


#for non-aug
# train accuracy: 0.9881840796019901
# test accuracy: 0.9613821138211383


In [ ]:
model.save('/kaggle/working/densenet.h5')
test_prediction=model.predict(test_x)
class_graph(test_prediction,test_y)

## nasnet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load NasNetMobile pre-trained on ImageNet data
base_model = NASNetMobile(weights='imagenet', include_top=False,input_shape=(224, 224, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully connected layers
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(train_x,train_y,epochs=35)

In [ ]:
prediction=model.predict(train_x)
test_prediction=model.predict(test_x)

evaluate(prediction,train_y,'train',False)
evaluate(test_prediction,test_y,'test',False)

# for non-aug
# train accuracy: 1.0
# test accuracy: 0.9451219512195121


In [ ]:
# model.save('/kaggle/working/nasnet.h5')
test_prediction=model.predict(test_x)
class_graph(test_prediction,test_y)